In [24]:
import cv2
import numpy as np

# Load the image in grayscale
image = cv2.imread('floorPlan_0.png', cv2.IMREAD_GRAYSCALE)


threshold = 60  # Adjust this value as needed

# Create a 2D matrix of the same size as the image
height, width = image.shape
result_matrix = np.zeros((height, width), dtype=np.int32)

# Fill the cells based on the threshold
for i in range(height):
    for j in range(width):
        if image[i, j] < threshold:
            result_matrix[i, j] = 0
        else:
            result_matrix[i, j] = -1

#draw_matrix(result_matrix)

In [25]:
import queue

def mark_pixel(x, y, value, pixel_queue):
    if x < 0 or y < 0 or x >= height or y >= width:
        return
    
    if result_matrix[x, y] != -1:
        return
    
    result_matrix[x, y] = value
    pixel_queue.put([x,y])

# Create a FIFO queue
pixel_queue = queue.Queue()

# enqueue all walls (v==0)

for i in range(height):
    for j in range(width):
        if result_matrix[i, j] == 0:
            pixel_queue.put([i, j])
            

# process each pixel
while not pixel_queue.empty():
    # print(pixel_queue.qsize(), " || ", i, "-", j)
    [i, j] = pixel_queue.get()
    current_value = result_matrix[i, j] + 1
    
    mark_pixel(i-1, j, current_value, pixel_queue)
    mark_pixel(i+1, j, current_value, pixel_queue)
    mark_pixel(i, j-1, current_value, pixel_queue)
    mark_pixel(i, j+1, current_value, pixel_queue)
    

print("done")

done


In [26]:
def draw_matrix(image):
    new_img = np.array(image.copy())
    # Convert -1 values to 255 (white) for visualization
    new_img[new_img == -1] = 127
    
    new_img = new_img.astype(np.uint8)

    print(new_img.shape)
    # Combine the mask with the original image to draw the cells
    cv2.imshow("Grayscale Image", new_img)
    cv2.waitKey(0)  # This line allows you to close the image window by pressing any key
    cv2.destroyAllWindows()

In [27]:
draw_matrix(result_matrix)

(818, 1124)


In [28]:
import random
import time


def random_color():
    red = np.int32(random.randint(150, 255))
    green = np.int32(random.randint(150, 255))
    blue = np.int32(random.randint(150, 255))
    return (red, green, blue)

def is_vaild_index(index, verify_painted_matrix):
    [x, y] = index
    num_rows = len(verify_painted_matrix)
    num_cols = len(verify_painted_matrix[0])
    
    if not (0 <= x < num_rows and 0 <= y < num_cols):
        return False
    
    if verify_painted_matrix[x][y] == True:
        return False
    
    # in range, not painted
    return True
    
def is_not_all_painted(verify_painted_matrix):
    for row in verify_painted_matrix:
        for value in row:
            if value == False:
                return True
            
    return False

def find_max(distance_matrix, verify_painted_matrix):
    max_val = 0
    cor = [-1, -1]
    
    for i in range(len(distance_matrix)):
        for j in range(len(distance_matrix[0])):
            if verify_painted_matrix[i][j] == False:
                # not paint
                if distance_matrix[i][j] > max_val:
                    max_val = distance_matrix[i][j]
                    cor = [i, j]
    
    return cor

def paint_room(cordinate, distance_matrix, verify_painted_matrix, colored_image):
    print("paint room of: ",cordinate)
    [x, y] = cordinate
    
    color = (0, 0, 0)
    if distance_matrix[x][y] == 2:
        color = (255, 255, 255)
    else:
        color = random_color()
        
    
    room_pixel_queue = queue.Queue()
    room_pixel_queue.put(cordinate)
    
    
    colored_image[x][y] = color
    verify_painted_matrix[x][y] = True
    count = 0
    while not room_pixel_queue.empty():
        [x, y] = room_pixel_queue.get()
        curent_value = distance_matrix[x][y]
        count = count + 1
        if count % 100000 == 0:
            print('.', end="")
        
        x_n = x -1
        y_n = y
        if is_vaild_index([x_n, y_n], verify_painted_matrix):
            if distance_matrix[x_n][y_n] <= curent_value:
                # able to paint
                colored_image[x_n][y_n] = color
                verify_painted_matrix[x_n][y_n] = True
                room_pixel_queue.put([x_n, y_n])
                
        x_n = x + 1
        y_n = y
        if is_vaild_index([x_n, y_n], verify_painted_matrix):
            if distance_matrix[x_n][y_n] <= curent_value:
                # able to paint
                colored_image[x_n][y_n] = color
                verify_painted_matrix[x_n][y_n] = True
                room_pixel_queue.put([x_n, y_n])
                
        x_n = x
        y_n = y - 1
        if is_vaild_index([x_n, y_n], verify_painted_matrix):
            if distance_matrix[x_n][y_n] <= curent_value:
                # able to paint
                colored_image[x_n][y_n] = color
                verify_painted_matrix[x_n][y_n] = True
                room_pixel_queue.put([x_n, y_n])
                
        x_n = x
        y_n = y + 1
        if is_vaild_index([x_n, y_n], verify_painted_matrix):
            if distance_matrix[x_n][y_n] <= curent_value:
                # able to paint
                colored_image[x_n][y_n] = color
                verify_painted_matrix[x_n][y_n] = True
                room_pixel_queue.put([x_n, y_n])
        

def create_colored_room(distance_matrix, colored_image):
    height = len(distance_matrix)
    width = len(distance_matrix[0])
    
    verify_painted_matrix = [[False for _ in range(width)] for _ in range(height)]
    
    # fill wall with 2
    for i in range(height):
        for j in range(width):
            if result_matrix[i][j] == 0:
                result_matrix[i][j] = 2
                
    while is_not_all_painted(verify_painted_matrix):
        # find max
        cordinate = find_max(distance_matrix, verify_painted_matrix)
        paint_room(cordinate, distance_matrix, verify_painted_matrix, colored_image)
        print("one room done")
        
    
    print("all room done")
    return colored_image
    
    # locate the higest value

In [29]:
print(result_matrix[538][235])
print(result_matrix[538][236])

165
165


In [30]:
colored_image = np.zeros((height, width, 3), dtype=np.uint8)
create_colored_room(result_matrix, colored_image)

print(colored_image.shape)
print(colored_image[0][0])

paint room of:  [538, 234]
.one room done
paint room of:  [206, 217]
.one room done
paint room of:  [206, 637]
.one room done
paint room of:  [456, 571]
one room done
paint room of:  [457, 570]
one room done
paint room of:  [458, 569]
one room done
paint room of:  [459, 568]
one room done
paint room of:  [460, 567]
one room done
paint room of:  [461, 566]
one room done
paint room of:  [462, 565]
one room done
paint room of:  [463, 564]
one room done
paint room of:  [464, 563]
one room done
paint room of:  [465, 562]
one room done
paint room of:  [466, 561]
one room done
paint room of:  [467, 560]
one room done
paint room of:  [468, 559]
one room done
paint room of:  [448, 388]
one room done
paint room of:  [466, 364]
one room done
paint room of:  [467, 363]
one room done
paint room of:  [468, 362]
one room done
paint room of:  [469, 361]
one room done
paint room of:  [470, 360]
one room done
paint room of:  [471, 359]
one room done
paint room of:  [472, 358]
one room done
paint room of

In [31]:
# Combine the mask with the original image to draw the cells
cv2.imshow("colored room Image", colored_image)
cv2.waitKey(0)  # This line allows you to close the image window by pressing any key
cv2.destroyAllWindows()

In [13]:
distance_matrix = [[1, 0, 0, 0, 0, 0, 0, 0], #0
                    [1, 0, 1, 1, 1, 1, 1, 0], #1
                    [1, 0, 1, 2, 2, 2, 1, 0], #2
                    [1, 0, 1, 2, 3, 2, 1, 0], #3
                    [1, 0, 1, 2, 3, 2, 1, 0], #4-4
                    [1, 0, 1, 1, 2, 1, 1, 0],
                    [1, 0, 0, 0, 1, 0, 0, 0],
                    [1, 1, 1, 1, 2, 1, 1, 0]]

verify_painted_matrix = [[False for _ in range(8)] for _ in range(8)]
verify_painted_matrix[2][3] = True

for i in range(8):
    for j in range(8):
        if distance_matrix[i][j] == 0:
            distance_matrix[i][j] = 2


print(find_max(distance_matrix, verify_painted_matrix))

colored_image = np.zeros((8, 8, 3), dtype=np.uint8)
create_colored_room(distance_matrix, colored_image)

#cv2.imshow("colored room Image", colored_image)
cv2.imwrite("rgb_image.png", colored_image)
#cv2.waitKey(0)  # This line allows you to close the image window by pressing any key
#cv2.destroyAllWindows()


[3, 4]
paint room of:  [3, 4]
one room done
paint room of:  [0, 1]
one room done
paint room of:  [7, 4]
one room done
all room done


True

In [7]:
import random
import time

def random_color():
    current_time = int(time.time())
    
    red = np.int32(random.randint(150, 255))
    green = np.int32(random.randint(150, 255))
    blue = np.int32(random.randint(150, 255))
    return (red, green, blue)


print(random_color())
print(random_color())
print(random_color())
print(random_color())

(186, 239, 159)
(163, 215, 167)
(166, 152, 191)
(227, 244, 201)


In [32]:

cv2.imwrite("rgb_image.png", colored_image)

True